In [1]:
import tensorflow as tf
import pickle
import numpy as np
import timeit
import time
import load_cifar
from scipy.sparse import csr_matrix
from load_cifar import features_reshape
from sklearn.utils import shuffle
from tensorflow.contrib.layers import flatten

In [2]:
tf.reset_default_graph()

In [3]:
#### EXAMPLE, FEEL FREE TO CHANGE #####
valid_features, valid_labels = pickle.load(open('vali_data.pickle','rb'))
training_features,training_labels = pickle.load(open('train_data.pickle','rb'))
test_features, test_labels = pickle.load(open('test_data.pickle','rb'))

In [4]:
valid_labels = np.squeeze(valid_labels.todense())
training_labels = np.squeeze(training_labels.todense())
test_labels = np.squeeze(test_labels.todense())

In [5]:
print(training_labels.shape)
print(training_labels[0])
print(training_features.shape)

(40000, 10)
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
(40000, 3072)


In [6]:
#print(training_features[0,0:100])

In [7]:
# reshape
training_features = features_reshape(training_features)
valid_features = features_reshape(valid_features)
test_features = features_reshape(test_features)
print(training_features.shape)
print(valid_features.shape)
print(test_features.shape)

(40000, 32, 32, 3)
(10000, 32, 32, 3)
(10000, 32, 32, 3)


<h1>Hyperparameter</h1>

In [8]:
EPOCHS = 10
BATCH_SIZE = 128
lr = 0.001

<h1>Placeholder</h1>

In [9]:
x = tf.placeholder(tf.float32, (None, 32, 32, 3))
y = tf.placeholder(tf.int32, (None, 10))
# one_hot_y = tf.one_hot(y, 10)

<h1>LeNet-5</h1>

In [10]:
def LeNet(x):    
    # Hyperparameters
    mu = 0
    sigma = 0.1
    layer_depth = {
        'layer_1' : 6,
        'layer_2' : 16,
        'layer_3' : 120,
        'layer_f1' : 84
    }

    
    # TODO: Layer 1: Convolutional. Input = 32x32x3. Output = 28x28x6.
    conv1_w = tf.Variable(tf.truncated_normal(shape = [5,5,3,6], mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(6))
    conv1 = tf.nn.conv2d(x,conv1_w, strides = [1,1,1,1], padding = 'VALID') + conv1_b 
    # TODO: Activation.
    conv1 = tf.nn.relu(conv1)

    # TODO: Pooling. Input = 28x28x6. Output = 14x14x6.
    pool_1 = tf.nn.max_pool(conv1, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID')
    
    # TODO: Layer 2: Convolutional. Output = 10x10x16.
    conv2_w = tf.Variable(tf.truncated_normal(shape = [5,5,6,16], mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(16))
    conv2 = tf.nn.conv2d(pool_1, conv2_w, strides = [1,1,1,1], padding = 'VALID') + conv2_b
    # TODO: Activation.
    conv2 = tf.nn.relu(conv2)

    # TODO: Pooling. Input = 10x10x16. Output = 5x5x16.
    pool_2 = tf.nn.max_pool(conv2, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID') 
    
    # TODO: Flatten. Input = 5x5x16. Output = 400.
    fc1 = flatten(pool_2)
    
    # TODO: Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1_w = tf.Variable(tf.truncated_normal(shape = (400,120), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(120))
    fc1 = tf.matmul(fc1,fc1_w) + fc1_b
    
    # TODO: Activation.
    fc1 = tf.nn.relu(fc1)

    # TODO: Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2_w = tf.Variable(tf.truncated_normal(shape = (120,84), mean = mu, stddev = sigma))
    fc2_b = tf.Variable(tf.zeros(84))
    fc2 = tf.matmul(fc1,fc2_w) + fc2_b
    # TODO: Activation.
    fc2 = tf.nn.relu(fc2)
    
    # TODO: Layer 5: Fully Connected. Input = 84. Output = 10.
    fc3_w = tf.Variable(tf.truncated_normal(shape = (84,10), mean = mu , stddev = sigma))
    fc3_b = tf.Variable(tf.zeros(10))
    predicted_logits = tf.matmul(fc2, fc3_w) + fc3_b
    return predicted_logits

<h1>Cost and Optimization</h1>

In [11]:
predicted_logits = LeNet(x)
loss_operation = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = predicted_logits, labels = y))
optimizer = tf.train.AdamOptimizer(learning_rate = lr)
training_operation = optimizer.minimize(loss_operation)

In [12]:
correct_prediction = tf.equal(tf.argmax(predicted_logits, 1), tf.argmax(y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#saver = tf.train.Saver()

<h1>Training, validation and testing</h1>
<h2>Train your model only 10 epochs.</h2>
<h2>1.Print out validation accuracy after each training epoch</h2>
<h2>2.Print out training time for each training epoch</h2>
<h2>3.Print out testing accuracy</h2>

In [13]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(training_features)
    #print(num_examples)
    print("Training...")
    for i in range(EPOCHS):
        start_time = time.time()
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = training_features[offset:end,:], training_labels[offset:end,:]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
        use_time = time.time() - start_time   
        #validation_accuracy = evaluate(x_validation, y_validation)
        validation_accuracy = sess.run(accuracy_operation,feed_dict={x:valid_features, y:valid_labels})
        print("EPOCH {0}, training time {1:.4f}s, Validation Accuracy = {2:.3f}".format(i+1, use_time, validation_accuracy))
        
    #saver.save(sess, './lenet')
    #print("Model saved")
    test_accuracy = sess.run(accuracy_operation,feed_dict={x:test_features, y:test_labels})
    print("Test Accuracy = {:.3f}".format(test_accuracy))

Training...
EPOCH 1, training time 4.1368s, Validation Accuracy = 0.432
EPOCH 2, training time 1.9031s, Validation Accuracy = 0.477
EPOCH 3, training time 2.0319s, Validation Accuracy = 0.508
EPOCH 4, training time 2.1108s, Validation Accuracy = 0.533
EPOCH 5, training time 2.0000s, Validation Accuracy = 0.550
EPOCH 6, training time 2.0489s, Validation Accuracy = 0.564
EPOCH 7, training time 1.9041s, Validation Accuracy = 0.569
EPOCH 8, training time 1.9011s, Validation Accuracy = 0.576
EPOCH 9, training time 1.8121s, Validation Accuracy = 0.580
EPOCH 10, training time 1.9473s, Validation Accuracy = 0.581
Test Accuracy = 0.580
